### import liberys needed

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from urllib.request import urlopen
import lxml.html as lh
import lxml
import ssl

### Read website

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bs(source, 'lxml')
print('soup ready')

soup ready


In [3]:
table = soup.find('table',{'class':'wikitable sortable'})

In [4]:
table_rows = table.find_all('tr')

### Import website to DataFrame

In [5]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

In [6]:
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


### Drop not asssingned borough

In [7]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

In [8]:
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### Join Duplicate posal codes

In [9]:
df= df.groupby('PostalCode').agg(lambda x: ','.join(x))

In [10]:
df.shape

(103, 2)

### All Neighbourhood that not assignd to be named the same as their borough

In [11]:
df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=df.loc[df['Neighbourhood']=="Not assigned",'Borough']
df=df.reset_index()

In [13]:
df.shape

(103, 3)